In [1]:
import sys, os
sys.path.append(os.getcwd() + '\..')
import pandas as pd
import numpy as np
from my_api import *
import matplotlib.pyplot as plt

0.9
12.685905874921438 8.685905874921438
0.5
15.15749916396364 11.15749916396364


d:\BaiduSyncdisk\Project\Co_simu_3\code_Cai2011\..\gen_S.py:139: ComplexWarning: Casting complex values to real discards the imaginary part
  S = S.astype(np.float64)


In [2]:
id = pd.MultiIndex.from_product([['fro', 2], [100, 300, 500]])
df = pd.DataFrame(index = id)

In [3]:
T = 300
rela = False
cov_str = 'Cai2011Adaptive_Model2_my' # 数据有些问题，随机数种子
# cov_str = 'Cai2011Adaptive_Model1'

In [4]:
simu_str = 'lx_band'
data_path = '../data_lxband_AR1_Grav_Cai/'
for N in [100, 300, 500]:
    if cov_str == 'Cai2011Adaptive_Model1':
        S = gen_S_Cai2011Adaptive_Model1(N)
    elif cov_str == 'Cai2011Adaptive_Model2_my':
        S = gen_S_Cai2011Adaptive_Model2_my(N = N, seed = 0)
    else:
        raise Exception
    
    for ord in ['fro', 2]:
        normS = np.linalg.norm(S, ord)
        
        for eta in [0.5, 0.8, 1]:
            cov_params = [cov_str]
            simu_params = [simu_str, eta]
            file_name = f"{ord},S,{N},{T},{cov_params},{simu_params}.txt"
            file_name = data_path + file_name
            data = np.loadtxt(file_name)

            col = str(simu_params)
            if col not in df.columns:
                df[col] = None
            if not rela:
                data *= normS # absolute error
            df[col][ord, N] = f"{data.mean() :.2f}({data.std() :.2f})"
df.head(1)

,,"['lx_band', 0.5]","['lx_band', 0.8]","['lx_band', 1]"
fro,100,7.68(0.38),4.57(0.26),4.17(0.20)


In [5]:
simu_str = 'LSRthreshold'
data_path = '../data_Cai2011_LSRthreshold_cv50/'
num_cv = 50

for N in [100, 300, 500]:
    if cov_str == 'Cai2011Adaptive_Model1':
        S = gen_S_Cai2011Adaptive_Model1(N)
    elif cov_str == 'Cai2011Adaptive_Model2_my':
        S = gen_S_Cai2011Adaptive_Model2_my(N = N, seed = 0)
    else:
        raise Exception
    
    for ord in ['fro', 2]:
        normS = np.linalg.norm(S, ord)
        
        for tau in [0.2]:
            # for prob in [0.99, 0.9, 1]:
            #     for qrob in [0, 0.01, 0.1]:
            for prob in [1, 0.99]:
                for qrob in [0, 0.01]:
                    cov_params = [cov_str]
                    simu_params = [simu_str, tau, prob, qrob, 'brute', num_cv]
                    file_name = f"{ord},S,{N},{T},{cov_params},{simu_params}.txt"
                    file_name = data_path + file_name
                    
                    try:
                        data = np.loadtxt(file_name)
                    except FileNotFoundError:
                        continue

                    col = str(simu_params)
                    if col not in df.columns:
                        df[col] = None
                    if not rela:
                        data *= normS # absolute error
                    df[col][ord, N] = f"{data.mean() :.2f}({data.std() :.2f})"
df.head(1)

,,"['lx_band', 0.5]","['lx_band', 0.8]","['lx_band', 1]","['LSRthreshold', 0.2, 1, 0, 'brute', 50]"
fro,100,7.68(0.38),4.57(0.26),4.17(0.20),10.83(1.56)


In [6]:
simu_str = '' # to be determined
data_path1 = '../data_OtherMethods_4-5/'
data_path2 = '../data_Cai2011_other_threshold/'

for N in [100, 300, 500]:
    if cov_str == 'Cai2011Adaptive_Model1':
        S = gen_S_Cai2011Adaptive_Model1(N)
    elif cov_str == 'Cai2011Adaptive_Model2_my':
        S = gen_S_Cai2011Adaptive_Model2_my(N = N, seed = 0)
    else:
        raise Exception
    
    for ord in ['fro', 2]:
        normS = np.linalg.norm(S, ord)

        # from Cai2011_other import Other_Methods
        for simu_str in ['Sample', 'Soft Threshold', 'Hard Threshold', 'Linear Shrink', 'Nonlinear Shrink']:
            data_path = data_path2 if 'Threshold' in simu_str else data_path1
            # print(data_path, simu_str)
 
            cov_params = [cov_str]
            simu_params = [simu_str]
            file_name = f"{ord},S,{N},{T},{cov_params},{simu_params}.txt"
            file_name = data_path + file_name
            try:
                data = np.loadtxt(file_name)
            except FileNotFoundError:
                continue

            col = str(*simu_params)
            if col not in df.columns:
                df[col] = None
            
            if not rela:
                data *= normS # absolute error
            df[col][ord, N] = f"{data.mean() :.2f}({data.std() :.2f})"
df.head(1)            

,,"['lx_band', 0.5]","['lx_band', 0.8]","['lx_band', 1]","['LSRthreshold', 0.2, 1, 0, 'brute', 50]",Sample,Soft Threshold,Hard Threshold,Linear Shrink,Nonlinear Shrink
fro,100,7.68(0.38),4.57(0.26),4.17(0.20),10.83(1.56),19.78(0.31),11.78(0.78),19.16(0.38),10.53(0.06),9.18(0.12)


In [7]:
df

['lx_band', 0.5] ['lx_band', 0.8] ['lx_band', 1]  \
fro 100       7.68(0.38)       4.57(0.26)     4.17(0.20)   
    300      21.06(0.34)      15.14(0.33)    13.79(0.33)   
    500      34.64(0.40)      26.38(0.42)    24.86(0.42)   
2   100       2.76(0.16)       1.62(0.16)     1.45(0.14)   
    300       3.99(0.14)       3.08(0.16)     3.07(0.18)   
    500       5.59(0.16)       4.06(0.15)     4.02(0.14)   

        ['LSRthreshold', 0.2, 1, 0, 'brute', 50]        Sample Soft Threshold  \
fro 100                              10.83(1.56)   19.78(0.31)    11.78(0.78)   
    300                              42.62(7.45)   76.88(0.47)    49.31(1.58)   
    500                             82.91(11.56)  144.40(0.68)    95.97(3.53)   
2   100                               3.47(0.62)    6.80(0.39)     3.37(0.30)   
    300                               7.64(1.41)   14.14(0.46)     8.67(0.47)   
    500                              13.32(1.70)   23.14(0.53)    14.87(0.72)   

        Hard Threshold Linear Shrink Nonlinear Shrink  
fro 100    19.16(0.38)   10.53(0.06)       9.18(0.12)  
    300    75.65(0.61)   29.25(0.07)             None  
    500   142.03(0.96)   50.67(0.08)             None  
2   100     5.76(0.35)    3.20(0.06)       2.49(0.09)  
    300    13.75(0.47)    7.06(0.15)             None  
    500    22.49(0.55)   11.39(0.19)             None

In [8]:
new_cols = []
for col in df.columns:
    try:
        col = eval(col)
    except Exception:
        pass

    if type(col) == str:
        new_col = col
    elif 'lx_band' in col:
        eta = col[1]
        new_col = fr'Augmented Band ($\eta={eta}$)'
    elif 'LSRthreshold' in col:
        t, p, q = col[1:-2]
        new_col = fr'Augmented Threshold ($\tau={t}, p={p}, q={q}$)' # The r in fr, means raw string. Otherwise I should write \\tau instead of \tau.
    new_cols.append(new_col)
df.columns = new_cols
df.head(1)

,,Augmented Band ($\eta=0.5$),Augmented Band ($\eta=0.8$),Augmented Band ($\eta=1$),"Augmented Threshold ($\tau=0.2, p=1, q=0$)",Sample,Soft Threshold,Hard Threshold,Linear Shrink,Nonlinear Shrink
fro,100,7.68(0.38),4.57(0.26),4.17(0.20),10.83(1.56),19.78(0.31),11.78(0.78),19.16(0.38),10.53(0.06),9.18(0.12)


In [9]:
buf = '../latex/' + cov_str + '.tex'
caption = cov_str

with pd.option_context("max_colwidth", 1000, 'display.width', 50):
    df.to_latex(buf = buf, 
                index = True, na_rep = ' ', sparsify = True, 
             column_format = 'll|' + 'p{2cm}' * df.shape[1], 
             longtable = False, 
             escape = False, encoding = 'utf-8', 
             caption = caption, label = "my label", 
             position = 'htbp')
# escape参数，防止字符被转换使得我设的数学模式被破坏

C:\Users\lx\AppData\Local\Temp\ipykernel_16592\2522360471.py:5: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df.to_latex(buf = buf,


In [10]:
pd.get_option('display.width')

80

In [11]:
lines = []

with open(buf, 'r') as fp:
    lines = fp.readlines()

s1 = r'\resizebox{\linewidth}{!}{' + '\n'
s2 = r'}' + '\n'
lines.insert(4, s1)
lines.insert(-1, s2)

with open(buf[:-4] + '_processed' + '.tex', 'w') as fp:
    fp.writelines(lines)
        # if not write_flag:
        #     if r'begin{tabular}' in line:
